In [0]:
df= spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/Volumes/pysparkdbt/source/source_data/customer/")

In [0]:
schema_customer=df.schema
schema_customer

StructType([StructField('customer_id', IntegerType(), True), StructField('first_name', StringType(), True), StructField('last_name', StringType(), True), StructField('email', StringType(), True), StructField('phone_number', StringType(), True), StructField('city', StringType(), True), StructField('signup_date', DateType(), True), StructField('last_updated_timestamp', TimestampType(), True)])

### ***PYSPARK STREAMING***

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS pysparkdbt.bronze.checkpoints;

In [0]:
entities=["customer","trips","location","payments","vehicles","drivers"]

In [0]:
for entity in entities:
    df_batch= spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(f"/Volumes/pysparkdbt/source/source_data/{entity}/")

    schema_entity=df_batch.schema

    df = spark.readStream.format("csv")\
    .option("header", "true")\
    .schema(schema_entity)\
    .load(f"/Volumes/pysparkdbt/source/source_data/{entity}/")
    df.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation",f"/Volumes/pysparkdbt/bronze/checkpoints/{entity}")\
    .trigger(once=True)\
    .toTable(f"pysparkdbt.bronze.{entity}")